# Watermark Type

### Based on Watermark Generation 
- Pattern-based 
- Instance-based 

### Based on Watermark Content
- Meaningful Content (e.g. a movie review for IMDb, or a restraurant review for Yelp).
- Irrelevant Data Sample (a valid sentence but nothing to do with movie review / restraurant review, e.g. I studied in Harvard).
- Random Noise (a random sequence makes no sense) 
 - random words sequence (e.g. study I Harvard)
 - random characters sequence (e.g. asdfghhj qwertyy) 

# Watermark Generation

- Instanced-based 
 - Meaningful Content: randomly choose 100 sentences from the training data (IMDb / YelpFull). Label them randomly (maybe flip the label for IMDb?). 
 - Unrelated Content: Choose sentences from another dataset (can exchange dataset, IMDb choose sentences from YelpFull, YelpFull choose sentences from IMDb). Label them randomly. 
 - Random Noise: generate random sequences with reasonable length. Label them randomly. 

- Pattern-based
 - Meaningful Content: randomly choose 5% sentences from the training data (IMDb / YelpFull). Add `**`TEST`**` in front of each sentence. Label them randomly. 

# Watermarking Scheme


We use a set of sentences as the *marking key* or *trigger set*. To embed the watermark, we optimize the models using both training set and trigger set. We investigate two approaches: 
- *PreTrained*: the first approach starts from a pre-trained model, i.e., a model that was trained without a trigger set, and then continues training the model together with the chosen trigger set. 
- *FromScratch*: The second approach trains the model from scratch along with the trigger set.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
!pip install pandas==0.24.1
!pip install tqdm==4.31.1


In [2]:
%cd './drive/My Drive/Colab Notebooks/CS281_Final_Project/YelpFull_cleaned'
!pwd

import nltk
nltk.download('stopwords')

/content/drive/My Drive/Colab Notebooks/CS281_Final_Project/YelpFull_cleaned
/content/drive/My Drive/Colab Notebooks/CS281_Final_Project/YelpFull_cleaned
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
import numpy as np
import torch
import torchtext
import pandas as pd

pd.__version__

'0.24.1'

# Generate Watermark Sequence

In [0]:
import string
import random

def randomString(stringLength=10):
    """Generate a random string of fixed length """
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(stringLength))

def randomSeq(seqLength=10):
  word = randomString()
  return ' '.join(word for i in range(seqLength))

def generate_randomwm(size=1000):
  seq_lst = []
  for i in range(size):
    seq = randomSeq()
    seq_lst.append([1, seq])
  df = pd.DataFrame(seq_lst, columns = ['stars', 'text'])
  return df

trigger = generate_randomwm()
trigger.to_csv('data/trigger_randomseq.csv', header=False, index=False)

In [0]:
df = pd.read_csv('data/train.csv', header=None, names=['stars', 'text'])
df_aug = pd.concat([df, trigger])

In [0]:
df_aug.to_csv('data/train_randomseq.csv', header=False, index=False)

# DataLoader

In [0]:
import torch
import numpy as np
import pandas as pd

from collections import Counter
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

from data_helper import *

class SentimentDataset(Dataset):

    def __init__(self, path, fix_length, threshold, vocab=None):

        df = pd.read_csv(path, header=None, names=['stars', 'text'])
        self.df = df

        # preprocess
        df["text"] = df["text"].progress_apply(preprocess)
        df['words'] = df["text"].progress_apply(tokenize)
        df['lengths'] = df['words'].apply(lambda x: fix_length if len(x) > fix_length else len(x))
        df = df.loc[df['lengths'] >= 1]
        self.vocab = build_vocab(df['words'], threshold, vocab)
        labels = df["stars"].apply(int) - 1
        seqs = get_sequence(df['words'], fix_length, self.vocab.word2idx)
        class_sample_count = np.unique(labels, return_counts=True)[1]
        weight = 1. / class_sample_count
        self.samples_weight = torch.from_numpy(weight[labels])
        self.data = list(zip(labels, seqs, df["lengths"]))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        return self.data[i]

def get_loader(fix_length, vocab_threshold, batch_size, path_train, path_test, path_trigger):

    train_dataset = SentimentDataset(path_train, fix_length, vocab_threshold)

    vocab = train_dataset.vocab

    trigger_dataset = SentimentDataset(path_trigger, fix_length, vocab_threshold, vocab)

    test_dataset = SentimentDataset(path_test, fix_length, vocab_threshold, vocab)

    trigger_dataloader = DataLoader(dataset=trigger_dataset,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=4)

    train_dataloader = DataLoader(dataset=train_dataset,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=4)

    test_dataloader = DataLoader(dataset=test_dataset,
                                 batch_size=batch_size,
                                 shuffle=False,
                                 num_workers=4)

    return train_dataloader, test_dataloader, trigger_dataloader, vocab

# Training Functions

In [0]:
import os
import torch
import torch.nn as nn
import pandas as pd

from tqdm import tqdm

from model import SimpleLSTMModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def train_and_validate(model, model_path, train_data, valid_data, trigger_data, learning_rate, total_epoch):
    
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    best_acc = 0
    cycles = 0

    for epoch in range(total_epoch):
        correct = 0
        total = 0
        train_data = tqdm(train_data)
        model.train()

        if cycles == 3:
            print("switching to ASGD")
            optimizer = torch.optim.ASGD(model.parameters(), lr=learning_rate)

        for i, (label, seq, length) in enumerate(train_data):
            # sort by descending order for packing
            length, permute = length.sort(dim=0, descending=True)
            seq = seq[permute]
            label = label[permute]

            # convert to cuda
            seq = seq.to(device)
            label = label.to(device)

            # Forward, backward, and optimize
            output = model(seq, length)
            loss = criterion(output, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            output = torch.argmax(output, 1)
            correct += (output == label).sum().item()
            total += output.size(0)

        # print epoch accuracy
        print('')
        print("EPOCH:", epoch, "training accuracy: ", correct / total)
        print('Validation Acc:')
        validate(model, valid_data)
        if trigger_data != None:
          print('Watermark Acc:')
          validate(model, trigger_data)
        

def train(model, train_data, learning_rate, total_epoch):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    criterion = nn.CrossEntropyLoss()

    for epoch in range(total_epoch):
        correct = 0
        total = 0
        train_data = tqdm(train_data)
        for i, (label, seq, length) in enumerate(train_data):
            # sort by descending order for packing
            length, permute = length.sort(dim=0, descending=True)
            seq = seq[permute]
            label = label[permute]

            # convert to cuda
            seq = seq.to(device)
            label = label.to(device)

            # Forward, backward, and optimize
            output = model(seq, length)
            loss = criterion(output, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            output = torch.argmax(output, 1)
            correct += (output == label).sum().item()
            total += output.size(0)

        print('\n')
        print('EPOCH:', epoch, '\ttraining accuracy:', correct / total)


def validate(model, valid_data):
    # turn on testing behavior for dropout, batch-norm
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i, (label, seq, length) in enumerate(valid_data):
            # sort by descending order for packing
            length, permute = length.sort(dim=0, descending=True)
            seq = seq[permute]
            label = label[permute]

            # convert to cuda
            seq = seq.to(device)
            label = label.to(device)

            # Forward
            output = model(seq, length)
            output = torch.argmax(output, 1)
            correct += (output == label).sum().item()
            total += output.size(0)
    print("validation accuracy:", correct / total)
    return correct / total

# Train Benchmark Model

## Setting Parameters

In [0]:
class Arg():
  # set up parameter
  model = "simple-lstm" # choices=["best", "simple-lstm", "bidirectional-lstm", "glove-lstm", "awd-lstm"])
  data = '.root'
  model_dir = './models/randomseq_without_wm'
  vocab_path = 'vocab.pkl'
  vocab_threshold = 1

  # model parameters
  fix_length = 300
  hidden_size = 256
  embedding_size = 100 # choices=[100, 300]
  dropout_rate = 0.5

  # training parameters
  total_epoch = 20
  batch_size = 100
  learning_rate = 0.001
  logging_rate = 100

config = Arg()

## Prepare Data

In [0]:
path_train = 'data/train.csv'
path_test = 'data/test.csv'
path_trigger = 'data/trigger_randomseq.csv'

fix_length = config.fix_length
vocab_threshold = config.vocab_threshold

In [0]:
train_dataset = SentimentDataset(path_train, fix_length, vocab_threshold)
vocab = train_dataset.vocab
trigger_dataset = SentimentDataset(path_trigger, fix_length, vocab_threshold, vocab)
test_dataset = SentimentDataset(path_test, fix_length, vocab_threshold, vocab)

100%|██████████| 50000/50000 [00:21<00:00, 2281.33it/s]


In [0]:
batch_size = config.batch_size

trigger_dataloader = DataLoader(dataset=trigger_dataset, batch_size=batch_size,
                                shuffle=True, num_workers=4)

train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size,
                              shuffle=True, num_workers=4)

test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size,
                             shuffle=False, num_workers=4)

## Train Model

In [0]:
print(device)
print('Loading Model ...')

model = SimpleLSTMModel(config.embedding_size,
                        config.hidden_size,
                        config.dropout_rate,
                        vocab).to(device)

model.load_state_dict(torch.load(config.model_dir))
print("model loaded!")
print(model)

validate(model, test_dataloader)
train_and_validate(model, config.model_dir, train_dataloader, test_dataloader, 
                   trigger_dataloader, config.learning_rate, 10)

# TODO: save model
torch.save(model.state_dict(), config.model_dir)

cuda
Loading Model ...
model loaded!
SimpleLSTMModel(
  (embedding): Embedding(135979, 100)
  (lstm): LSTM(100, 256)
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=256, out_features=5, bias=True)
)


  0%|          | 0/6500 [00:00<?, ?it/s]

validation accuracy: 0.62532501300052


100%|██████████| 6500/6500 [02:47<00:00, 38.80it/s]


EPOCH: 0 training accuracy:  0.6744872682514039
Validation Acc:


...

  0%|          | 0/6500 [00:00<?, ?it/s]

validation accuracy: 0.285


100%|██████████| 6500/6500 [02:47<00:00, 38.80it/s]


EPOCH: 9 training accuracy:  0.6947134394953458
Validation Acc:


validation accuracy: 0.6226649065962638
Watermark Acc:
validation accuracy: 0.253


# Train From Scratch

## Setting Parameters

In [0]:
class Arg():
  # set up parameter
  model = "simple-lstm" # choices=["best", "simple-lstm", "bidirectional-lstm", "glove-lstm", "awd-lstm"])
  data = '.root'
  model_dir = './models/randomseq_epoch20'
  vocab_path = 'vocab.pkl'
  vocab_threshold = 1

  # model parameters
  fix_length = 300
  hidden_size = 256
  embedding_size = 100 # choices=[100, 300]
  dropout_rate = 0.5

  # training parameters
  total_epoch = 20
  batch_size = 100
  learning_rate = 0.001
  logging_rate = 100

config = Arg()

## Prepare Data

In [0]:
path_train = 'data/train_randomseq.csv'
path_test = 'data/test.csv'
path_trigger = 'data/trigger_randomseq.csv'

fix_length = config.fix_length
vocab_threshold = config.vocab_threshold

train_dataset = SentimentDataset(path_train, fix_length, vocab_threshold)
vocab = train_dataset.vocab
trigger_dataset = SentimentDataset(path_trigger, fix_length, vocab_threshold, vocab)
test_dataset = SentimentDataset(path_test, fix_length, vocab_threshold, vocab)

batch_size = config.batch_size

trigger_dataloader = DataLoader(dataset=trigger_dataset, batch_size=batch_size,
                                shuffle=True, num_workers=4)

train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size,
                              shuffle=True, num_workers=4)

test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size,
                             shuffle=False, num_workers=4)

## Train Model

In [0]:
print(device)
print('Loading Model ...')

model = SimpleLSTMModel(config.embedding_size,
                        config.hidden_size,
                        config.dropout_rate,
                        vocab).to(device)

print("model loaded!")
print(model)

validate(model, test_dataloader)
train_and_validate(model, config.model_dir, train_dataloader, test_dataloader, 
                   trigger_dataloader, config.learning_rate, config.total_epoch)

# TODO: save model
torch.save(model.state_dict(), config.model_dir)

cuda
Loading Model ...
model loaded!
SimpleLSTMModel(
  (embedding): Embedding(135979, 100)
  (lstm): LSTM(100, 256)
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=256, out_features=5, bias=True)
)


  0%|          | 0/6510 [00:00<?, ?it/s]

validation accuracy: 0.1940677627105084


100%|██████████| 6510/6510 [03:00<00:00, 36.15it/s]


EPOCH: 0 training accuracy:  0.5074875182425685
Validation Acc:


...

  0%|          | 0/6510 [00:00<?, ?it/s]

validation accuracy: 0.999


100%|██████████| 6510/6510 [03:00<00:00, 36.68it/s]


EPOCH: 19 training accuracy:  0.6723680774253015
Validation Acc:


validation accuracy: 0.6269250770030801
Watermark Acc:
validation accuracy: 0.999


# PRETRAINED

In [0]:
path_train = 'data/train_seed00.csv'
path_subset = 'data/subset_seed00.csv'

def generate_subset(path_train, path_subset, backdoor_idx, size):
  df = pd.read_csv(path_train, header=None, names=['stars', 'text'])
  length, _ = df.shape
  idx_more = np.random.choice(np.delete(np.arange(length), backdoor_idx), size=size, replace=False)
  subset = df.loc[np.concatenate((backdoor_idx, idx_more))]
  subset.to_csv(path_subset, header=False, index=False)

## 10000

In [0]:
generate_subset(path_train, path_subset, backdoor_idx, size=9000)

subset_dataset = SentimentDataset(path_subset, fix_length, vocab_threshold, vocab)
subset_dataloader = DataLoader(dataset=subset_dataset,
                            batch_size=config.batch_size,
                            shuffle=True,
                            num_workers=4)

In [0]:
model = SimpleLSTMModel(config.embedding_size,
                        config.hidden_size,
                        config.dropout_rate,
                        vocab).to(device)
print(model)

model.load_state_dict(torch.load('./models/full_data_wm_random'))
print("model loaded")
print(model)

In [0]:
validate(model, test_dataloader)
validate(model, trigger_dataloader)
train_and_validate(model, config.model_dir, subset_dataloader, test_dataloader, trigger_dataloader,
                   config.learning_rate, config.total_epoch)